In [1]:
from typing import List, Tuple
from scipy.stats import _stats_py
import numpy as np
import torch
from torch import Tensor
from scipy.optimize import linear_sum_assignment
from scipy.ndimage import center_of_mass

from data.MoNuSeg.dataset import MoNuSeg
from data.MoNuSeg.data_module import MoNuSegDataModule
from augmentation.augmentations import *
from evaluation.utils import tensor_intersection, tensor_union, intersection_over_union
from data.MoNuSeg.ground_truth import NucleiInstances
from data.MoNuSeg.illustrator import Picture
from data.MoNuSeg.utils import get_bbox, cuda_tensor_to_ndarray
from data.MoNuSeg.ground_truth import NucleiInstances

/home/ubuntu/NucInstSeg/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def welch(m1: float, m2: float,v1: float, v2: float, n1: int, n2: int, alternative: str="two-sided") -> float:
    """
    Welch t-test

    The Welch t-test assumes normal-distributed populations with unequal variances.

    alternative : {'two-sided', 'less', 'greater'}, optional
    """
    df, denom = _stats_py._unequal_var_ttest_denom(v1, n1, v2, n2)
    return _stats_py._ttest_ind_from_stats(m1, m2, denom, df, alternative)[1]

In [3]:
def setup():
    data_module = MoNuSegDataModule(
    seg_masks=True,
    cont_masks=True,
    dist_maps=True,
    hv_maps=True,
    labels=False,
    data_root="datasets"
    )

    data_module.setup(stage="test")
    loader = data_module.test_dataloader()
    batch = next(iter(loader))



    i = 7
    return [gt[i] for gt in batch.values()]

In [4]:
img, seg_mask, cont_mask, dist_map, gt, hv_map = setup()
seg_mask = cuda_tensor_to_ndarray(seg_mask)
dist_map = cuda_tensor_to_ndarray(dist_map)

In [5]:
def fn0():
    default = {
        "a": 0,
        "b": 1,
        "c": 2
    }
    new = {
        "d": 3,
        "e": 4
    }
    return {**default, **new}

In [14]:
def fn1():
    default = {
        "a": 0,
        "b": 1,
        "c": 2
    }
    new = {
        "d": 3,
        "e": 4
    }
    return new.update(default)

In [17]:
%timeit -r 20 -n 100000 -p 5 fn0()

261.53 ns ± 13.809 ns per loop (mean ± std. dev. of 200 runs, 100,000 loops each)


In [18]:
%timeit -r 20 -n 100000 -p 5 fn1()

265.77 ns ± 1.7601 ns per loop (mean ± std. dev. of 200 runs, 100,000 loops each)


In [ ]:
welch(m1=14.779,m2=13.157,v1=0.407,v2=0.146,n1=20,n2=20)

In [14]:
fn0(dist_map).dtype

dtype('int32')